# 2. AI Text Generation (Generación de Texto Artificial)

* **Objetivo:** Crear el "Dataset Sombra" (Shadow Dataset).
* **Estrategia:** Utilizar la API de **Google Gemini** para reescribir cada noticia del dataset humano original. El objetivo es mantener los hechos informativos pero alterar el estilo sintáctico y léxico para que refleje los patrones de una IA.

## 2.1. Configuración e Importaciones

Importamos las librerías necesarias y definimos las constantes globales.
* Definimos `INPUT_FILE` (la muestra humana creada en el Notebook 1) y `OUTPUT_FILE` (donde se guardará el dataset pareado de IA).

In [ ]:
#%pip install google-generativeai

  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_api_core-2.28.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached google_api_python_client-2.187.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached google_auth-2.43.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.72.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached grpcio_status-1.76.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Using cached grpcio_status-1.75.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_status-1.75.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_status-1.74.0-

In [ ]:
from google import generativeai as genai
import pandas as pd
import time
import os

# API Key de Google Gemini (PERSONAL)
GOOGLE_API_KEY = "AIza....."

# Rutas de archivos
INPUT_FILE = '../data/1_raw/all-the-news-5k-sample.csv'
OUTPUT_FILE = '../data/3_synthetic/ai_generated_gemini.csv'

# Crear carpeta de salida si no existe
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

## 2.2. Inicialización del Cliente Gemini

Configuramos el cliente con la clave API y seleccionamos el modelo.
Usamos **`gemini-2.0-flash`**, **`gemini-2.5-flash-lite-preview-09-2025`** por ser el modelo más eficiente (rápido y gratuito) para tareas de reescritura masiva.

In [63]:
# Configurar la API
genai.configure(api_key=GOOGLE_API_KEY)

# Instanciar el modelo
model = genai.GenerativeModel('gemini-2.5-flash-lite-preview-09-2025')

print("✅ Cliente Gemini configurado correctamente.")

✅ Cliente Gemini configurado correctamente.


## 2.3. Definición de la Función de Reescritura

Definimos la función `rewrite_article`.
* **Prompt Engineering:** Instruimos al modelo para que actúe como un "Periodista IA". Le pedimos explícitamente que mantenga la información pero use su propio estilo.
* **Limpieza:** Solicitamos que no añada introducciones ("Here is the text..."), solo el cuerpo del artículo.

In [10]:
def rewrite_article(text):
    """
    Envía el texto a Gemini para ser reescrito con estilo sintético.
    """
    # Prompt diseñado para imitar estilo periodístico artificial
    prompt = f"""
    Act as an AI journalist. Rewrite the following news article text.
    Maintain the core information and facts, but use your own sentence structure and vocabulary.
    Do not output any introduction like "Here is the rewritten text". Output ONLY the article body.
    
    Original Text:
    {text[:8000]} 
    """
    
    # Generación de contenido
    response = model.generate_content(prompt)
    
    # Retornamos el texto limpio
    return response.text.strip()

## 2.4. Bucle Principal de Generación

Iteramos sobre el dataset humano.
1. Leemos el artículo original.
2. Lo enviamos a Gemini.
3. Guardamos el resultado en una lista.
4. Aplicamos un `time.sleep` para respetar los límites de velocidad de la versión gratuita de la API.

Nota: Algunos de los artículos (aprox. el 0.1%) no pueden ser reescritos por el LLM debido a la violación de políticas de contenido al tratarse de artículos de índole sexual o violenta. Sin embargo, no supone ningún problema, se filtrarán en el notebook 3.

In [64]:
## 2.4. Bucle Principal (Con Diagnóstico)
import os

print("🚀 Cargando datos humanos...")
if not os.path.exists(INPUT_FILE):
    print(f"❌ Error CRÍTICO: No encuentro el archivo {INPUT_FILE}")
else:
    df_human = pd.read_csv(INPUT_FILE)
    total_filas = len(df_human)
    print(f"📊 Total de artículos a procesar: {total_filas}")

    # -----------------------------------------------------------
    # LÓGICA DE REANUDACIÓN
    # -----------------------------------------------------------
    processed_ids = set()
    if os.path.exists(OUTPUT_FILE):
        try:
            existing = pd.read_csv(OUTPUT_FILE)
            # Solo si el archivo tiene datos y la columna correcta
            if 'original_id' in existing.columns:
                processed_ids = set(existing['original_id'].unique())
                print(f"🔄 ARCHIVO DETECTADO: Ya existen {len(processed_ids)} artículos procesados.")
            else:
                print("⚠️ El archivo de salida existe pero no tiene la estructura correcta.")
        except Exception as e:
            print(f"⚠️ Error leyendo archivo existente (se ignorará): {e}")

    # COMPROBACIÓN DE SEGURIDAD
    if len(processed_ids) >= total_filas:
        print("\n🛑 ¡ATENCIÓN! El script se ha detenido porque PARECE QUE YA ACABASTE.")
        print(f"   -> Filas totales: {total_filas}")
        print(f"   -> Filas ya procesadas: {len(processed_ids)}")
        print("   ✅ SOLUCIÓN: Si quieres empezar de cero, BORRA el archivo:")
        print(f"   rm {OUTPUT_FILE}")
    
    else:
        print(f"▶️ Iniciando trabajo... Faltan {total_filas - len(processed_ids)} artículos.")

        # Header mode: Solo escribimos cabecera si el archivo NO existe
        header_mode = not os.path.exists(OUTPUT_FILE)

        for index, row in df_human.iterrows():
            # 1. SALTAR YA PROCESADOS
            if index in processed_ids:
                continue

            original_text = row['article']
            
            try:
                # 2. LLAMADA A LA API
                ai_text = rewrite_article(original_text)
                
                if ai_text:
                    # 3. GUARDADO INCREMENTAL
                    single_row = pd.DataFrame([{
                        'original_id': index,
                        'article': ai_text,
                        'label': 1
                    }])
                    
                    single_row.to_csv(OUTPUT_FILE, mode='a', header=header_mode, index=False)
                    header_mode = False 
                    
            except Exception as e:
                print(f"⚠️ Error en artículo {index}: {e}")

            # 4. PAUSA (IMPORTANTE)
            time.sleep(4)
            
            # Feedback visual
            if (index + 1) % 10 == 0:
                print(f"   ... Procesados {index + 1}/{total_filas}")

        print(f"\n✅ ¡Proceso finalizado! Datos en: {OUTPUT_FILE}")

🚀 Cargando datos humanos...
📊 Total de artículos a procesar: 5000
🔄 ARCHIVO DETECTADO: Ya existen 1346 artículos procesados.
▶️ Iniciando trabajo... Faltan 3654 artículos.
   ... Procesados 1350/5000
   ... Procesados 1360/5000
   ... Procesados 1370/5000
   ... Procesados 1380/5000
   ... Procesados 1390/5000
   ... Procesados 1400/5000
   ... Procesados 1410/5000
   ... Procesados 1420/5000
   ... Procesados 1430/5000
   ... Procesados 1440/5000
   ... Procesados 1450/5000
   ... Procesados 1460/5000
   ... Procesados 1470/5000
   ... Procesados 1480/5000
   ... Procesados 1490/5000
   ... Procesados 1500/5000
   ... Procesados 1510/5000
   ... Procesados 1520/5000
   ... Procesados 1530/5000
   ... Procesados 1540/5000
   ... Procesados 1550/5000
   ... Procesados 1560/5000
   ... Procesados 1570/5000
   ... Procesados 1580/5000
   ... Procesados 1590/5000
   ... Procesados 1600/5000
   ... Procesados 1610/5000
   ... Procesados 1620/5000
   ... Procesados 1630/5000
   ... Procesado

KeyboardInterrupt: 

In [48]:
# SCRIPT PARA RECUPERAR Y GUARDAR EL ARTÍCULO 1159
import pandas as pd
import os

index_to_check = 1287
row_1159 = df_human.iloc[index_to_check]

print(f"🧐 Intentando recuperar artículo {index_to_check}...")

try:
    ai_text = rewrite_article(row_1159['article'])
    
    if ai_text:
        print("✅ ¡ÉXITO! Generado correctamente.")
        
        # PREPARAMOS EL GUARDADO
        single_row = pd.DataFrame([{
            'original_id': index_to_check,
            'article': ai_text,
            'label': 1
        }])
        
        # GUARDAMOS EN EL CSV (Append)
        # Nota: Asumimos que el archivo ya existe y tiene cabeceras
        single_row.to_csv(OUTPUT_FILE, mode='a', header=False, index=False)
        
        print(f"💾 ¡GUARDADO! El artículo {index_to_check} se ha añadido al final de {OUTPUT_FILE}")
        
    else:
        print("❌ FALLO: Gemini se niega a procesarlo (posible filtro de seguridad).")

except Exception as e:
    print(f"❌ ERROR TÉCNICO: {e}")

🧐 Intentando recuperar artículo 1287...
❌ ERROR TÉCNICO: Invalid operation: The `response.parts` quick accessor requires a single candidate, but but `response.candidates` is empty.
This appears to be caused by a blocked prompt, see `response.prompt_feedback`: block_reason: PROHIBITED_CONTENT



In [36]:
# CHECK LIST OF MODELS AVAILABLE
models = genai.list_models()
for m in models:
    print(m)

Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representation of a text.',
      input_token_limit=1024,
      output_token_limit=1,
      supported_generation_methods=['embedText', 'countTextTokens'],
      temperature=None,
      max_temperature=None,
      top_p=None,
      top_k=None)
Model(name='models/gemini-2.5-pro-preview-03-25',
      base_model_id='',
      version='2.5-preview-03-25',
      display_name='Gemini 2.5 Pro Preview 03-25',
      description='Gemini 2.5 Pro Preview 03-25',
      input_token_limit=1048576,
      output_token_limit=65536,
      supported_generation_methods=['generateContent',
                                    'countTokens',
                                    'createCachedContent',
                                    'batchGenerateContent'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)
Model(na